<a href="https://colab.research.google.com/github/emnordi/ce888/blob/master/lab5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [0]:
data = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv", header=None)
data = data.drop(data.columns[0], axis=1)

In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
arr = data.values
rated = np.where(arr!=99)
newdata, tup = replace(data.values)

In [0]:
x, y = tup

In [0]:
#LATENT FACTOR

In [0]:
n_latent_factors = 6

user_ratings = newdata
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:

def predict_rating(joke_id, rating_id):
    """ Predict a rating given a joke_id and an rating_id.
    """
    user_preference = latent_user_preferences[joke_id]
    item_preference = latent_item_features[rating_id]
    return user_preference.dot(item_preference)


def train(joke_id, rating_id, rating, alpha=0.0001):
    
    #print rating_id
    prediction_rating = predict_rating(joke_id, rating_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[joke_id][:]
    latent_user_preferences[joke_id] -= alpha * err * latent_item_features[rating_id]
    latent_item_features[rating_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=50):
    """ Iterate over all jokes and all ratings and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for joke_id in range(latent_user_preferences.shape[0]):
            for rating_id in range(latent_item_features.shape[0]):
                rating = user_ratings[joke_id][rating_id]
                if rating != 99:
                    err = train(joke_id, rating_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10) == 0:
            print(mse)

In [9]:
sgd()

25.572119530396208
24.206648932556515
21.54373842800415
17.927381281465355
17.494832892131342


In [0]:
predictions = latent_user_preferences.dot(latent_item_features.T)

In [0]:
error = 0

In [0]:
for val in range(len(x)):
  error += (data.values[x[val], y[val]] - predictions[x[val], y[val]])**2


In [157]:
print("MSE 50 iter, alpha=0.0001, latent = 2:", error/len(x))

MSE: 18.17020494399254


In [189]:
print("MSE 70 iter, alpha=0.0001, latent = 2:", error/len(x))

MSE 70 iter, alpha=0.0001: 18.299726039034176


In [41]:
print("MSE 50 iter, alpha=0.00001, latent = 2:", error/len(x))

MSE 50 iter, alpha=0.00001: 24.554725415725215


In [52]:
print("MSE 50 iter, alpha=0.0001, latent = 3:", error/len(x))

MSE 50 iter, alpha=0.0001, latent = 3: 18.014014935812405


In [59]:
print("MSE 50 iter, alpha=0.0001, latent = 4:", error/len(x))

MSE 50 iter, alpha=0.0001, latent = 4: 17.975855481689663


In [77]:
print("MSE 50 iter, alpha=0.0001, latent = 5:", error/len(x))

MSE 50 iter, alpha=0.0001, latent = 5: 17.248999747688927


In [16]:
print("MSE 50 iter, alpha=0.0001, latent = 6:", error/len(x))

MSE 50 iter, alpha=0.0001, latent = 6: 18.06304370488036
